In [1]:
import pandas as pd
import numpy as np

## Read in excel files


In [2]:
dot_df = pd.read_excel("dot_hnl_roads.xls")

In [3]:
dot_df

,OBJECTID,year_recor,state_code,route_id,begin_poin,end_point,f_system,urban_code,county_cod,route_numb,route_sign,alternativ,speed_limi,terrain_ty,widening_o,SHAPE__Len,Shape_Leng
0,1,2019,15,HI2P35,0.100,0.166,7,89770,3,0,10,Alakawa Street (Honolulu Harbor Pier 35),0,1,A,0.001045,113.918507
1,2,2019,15,HI2P35,0.166,0.183,7,89770,3,0,10,Alakawa Street (Honolulu Harbor Pier 35),0,1,A,0.000269,29.342661
2,3,2019,15,93,7.400,7.490,3,89770,3,93,4,Farrington Highway,35,2,AFG,0.001385,144.870529
3,4,2019,15,93,7.490,7.500,3,89770,3,93,4,Farrington Highway,35,2,DFG,0.000153,16.096705
4,5,2019,15,93,13.083,13.090,3,89770,3,93,4,Farrington Highway,25,1,A,0.000104,11.267835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65530,65531,2019,15,930,4.431,4.465,4,99998,3,930,4,Farrington Highway,45,1,X,0.000526,54.690719
65531,65532,2019,15,930,0.600,0.700,4,99999,3,930,4,Farrington Highway,45,1,BF,0.001553,160.858245
65532,65533,2019,15,19,88.200,88.300,3,99998,1,19,4,Queen Kaahumanu Highway,55,2,G,0.001484,160.787439
65533,65534,2019,15,19,36.770,36.780,3,99999,1,19,4,Hawaii Belt Road,55,2,D,0.000153,16.113928


## Clean DOT State road data
Try aggregating speeds (mean) for roads with same name 

In [6]:
dot_df["speed_limi"].dtype

dtype('int64')

In [7]:
dot_df["speed_limi"].unique()

array([ 0, 35, 25, 55, 20, 30, 45, 15, 60, 50, 40,  5, 10])

In [8]:
# Filter out those rows without speed limit (speed limit = 0)

dot_spd = dot_df[dot_df["speed_limi"] != 0]
dot_spd.shape

(64520, 17)

In [9]:
# Save those with only one speed limit
# This is ready for join

dot_spd_rd1 = dot_spd[~dot_spd["alternativ"].duplicated(keep = False)]
dot_spd_rd1.shape

(1, 17)

In [10]:
dot_spd_rd1 = dot_spd_rd1[["alternativ", "speed_limi"]]    # Only road with no duplicates
dot_spd_rd1 = dot_spd_rd1.rename(columns = {"speed_limi" : "avg_spd"})

In [28]:
dot_spd_rd1 = dot_spd_rd1.rename(columns={"alternativ": "road_name"})

In [29]:
dot_spd_rd1

,road_name,avg_spd
21417,Pakini Street,55


In [12]:
# Grab rows with duplicate names

dot_spd_dupl = dot_spd[dot_spd["alternativ"].duplicated(keep = False)]
dot_spd_dupl.shape

(64519, 17)

In [13]:
# Number of unique names in the duplicated subset

len(dot_spd_dupl["alternativ"].unique())

631

In [14]:
# Create new column that sums total length of particular road

dot_spd_dupl["tot_len"] = dot_spd_dupl.groupby("alternativ")["Shape_Leng"].transform("sum")

/var/folders/7z/bh_vxcz94c17ntk9rz3rrq3h0000gn/T/ipykernel_12553/2156704658.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dot_spd_dupl["tot_len"] = dot_spd_dupl.groupby("alternativ")["Shape_Leng"].transform("sum")


In [15]:
# Check Farrington Highway example

dot_spd_dupl[dot_spd_dupl["alternativ"] == "Farrington Highway"]

,OBJECTID,year_recor,state_code,route_id,begin_poin,end_point,f_system,urban_code,county_cod,route_numb,route_sign,alternativ,speed_limi,terrain_ty,widening_o,SHAPE__Len,Shape_Leng,tot_len
2,3,2019,15,93,7.400,7.490,3,89770,3,93,4,Farrington Highway,35,2,AFG,0.001385,144.870529,48096.66842
3,4,2019,15,93,7.490,7.500,3,89770,3,93,4,Farrington Highway,35,2,DFG,0.000153,16.096705,48096.66842
4,5,2019,15,93,13.083,13.090,3,89770,3,93,4,Farrington Highway,25,1,A,0.000104,11.267835,48096.66842
5,6,2019,15,93,6.450,6.500,3,89770,3,93,4,Farrington Highway,35,1,AFG,0.000752,80.483364,48096.66842
22,23,2019,15,93,18.900,18.990,4,99999,3,93,4,Farrington Highway,25,3,DF,0.001363,144.872760,48096.66842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65519,65520,2019,15,93,15.900,16.000,3,89770,3,93,4,Farrington Highway,25,1,FG,0.001456,160.969819,48096.66842
65520,65521,2019,15,93,4.062,4.080,3,89770,3,93,4,Farrington Highway,35,2,DFG,0.000266,28.973844,48096.66842
65529,65530,2019,15,930,1.838,1.840,4,99999,3,930,4,Farrington Highway,45,1,B,0.000031,3.217105,48096.66842
65530,65531,2019,15,930,4.431,4.465,4,99998,3,930,4,Farrington Highway,45,1,X,0.000526,54.690719,48096.66842


In [16]:
# Verify Sum operation works (yes)

dot_spd_dupl[dot_spd_dupl["alternativ"] == "Farrington Highway"]["Shape_Leng"].sum()    

48096.66842042876

In [17]:
dot_spd_dupl["agg_spd"] = (dot_spd_dupl["speed_limi"] * dot_spd_dupl["Shape_Leng"]) / dot_spd_dupl["tot_len"]
dot_spd_dupl["avg_spd"] = dot_spd_dupl.groupby("alternativ")["agg_spd"].transform("sum")

/var/folders/7z/bh_vxcz94c17ntk9rz3rrq3h0000gn/T/ipykernel_12553/3548215244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dot_spd_dupl["agg_spd"] = (dot_spd_dupl["speed_limi"] * dot_spd_dupl["Shape_Leng"]) / dot_spd_dupl["tot_len"]


In [23]:
dot_spd_rd2 = dot_spd_dupl[["alternativ", "avg_spd"]].drop_duplicates()
dot_spd_rd2 = dot_spd_rd2.rename(columns={"alternativ": "road_name"})

In [27]:
dot_spd_rd2

,road_name,avg_spd
2,Farrington Highway,33.640771
6,Joseph P Leong Highway,43.177286
7,Kamehameha Highway,35.413415
8,Front Street,20.696062
9,Kapoho Kalapana Road,30.846234
...,...,...
49868,Pier 2 access road (Port of Kawaihae),48.999974
52858,Kao Road,35.000000
54249,H-1_WB_7,55.000000
58000,Saratoga Road,45.000000


In [30]:
dot_spd_comb = pd.concat([dot_spd_rd2, dot_spd_rd1])
dot_spd_comb

,road_name,avg_spd
2,Farrington Highway,33.640771
6,Joseph P Leong Highway,43.177286
7,Kamehameha Highway,35.413415
8,Front Street,20.696062
9,Kapoho Kalapana Road,30.846234
...,...,...
52858,Kao Road,35.000000
54249,H-1_WB_7,55.000000
58000,Saratoga Road,45.000000
65184,Ala Luina Street (Port of Kahului),7.010062


In [31]:
# Get rid of one NA row
dot_spd_notna = dot_spd_comb[~dot_spd_comb["road_name"].isna()]

In [33]:
dot_spd_notna.to_csv("dot_spd_comb.csv", index= False)

## Experimental

In [223]:
cmn_name = ["Farrington"]

In [224]:
mask = df["fullname"].apply(lambda x: any(item for item in cmn_name if item in x))

TypeError: argument of type 'float' is not iterable

In [19]:
df1 = df[mask]
df1

,objectid,segmentid,fullname,class,street_cla,zipcoder,zipcodel,townl,townr,neighborr,oneway,edittype,revisedate,owner,loaddate,st_lengths,speed
37,38,8882,FARRINGTON HWY,A21,8,96792.0,96792.0,Waianae,Waianae,WAIANAE,0,NaN,2003-05-21,STATE,2009-05-15,440.395735,0
226,227,9954,FARRINGTON HWY,A21,8,96797.0,96797.0,Waipahu,Waipahu,WAIPAHU,0,NaN,2003-05-21,STATE,2009-05-15,68.583003,0
422,423,1009,FARRINGTON HWY,A21,8,96791.0,96791.0,Waialua,Waialua,NORTH SHORE,0,NaN,2003-05-21,STATE,2009-05-15,351.295967,0
595,596,5854,FARRINGTON HWY,A21,8,96792.0,96792.0,Waianae,Waianae,WAIANAE,0,NaN,2003-05-21,STATE,2009-05-15,73.247810,0
640,641,8288,FARRINGTON HWY,A25,8,96782.0,96782.0,Pearl City,Pearl City,PEARL CITY,0,NaN,2003-05-21,VARIOUS,2009-05-15,445.009136,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24743,24744,13800,FARRINGTON HWY,A21,8,96707.0,96707.0,Waianae,Kapolei,MAKAKILO/KAPOLEI/HONOKAI HALE,0,NaN,2015-03-06,STATE,2015-03-06,235.429850,0
24766,24767,13778,FARRINGTON HWY,A21,8,96707.0,96707.0,Waianae,Kapolei,MAKAKILO/KAPOLEI/HONOKAI HALE,0,NaN,2015-03-06,STATE,2015-03-06,911.608356,0
24792,24793,10396,FARRINGTON HWY,A21,8,96797.0,96797.0,Ewa Beach,Ewa Beach,WAIPAHU,1,NaN,2015-03-06,STATE,2015-03-06,111.850830,0
24839,24840,8532,FARRINGTON HWY,A21,8,96782.0,96782.0,Waipahu,Waipahu,WAIPAHU,1,NaN,2015-03-06,VARIOUS,2015-03-06,60.596824,0
